In [1]:
import exos.streams
from exos.streams.generator import stream_producer
from exos.streams.generator import stream_consumer
from exos.streams.simulator import run_experiment

from multiprocessing import Process, Queue, Condition

from skmultiflow.data import TemporalDataStream

import numpy as np

In [2]:
data1 = np.array([[1,2],[3,4],[5,6],[7,8], [9,10]])
data2 = np.array([[11,12,101],[13,14,102],[15,16,103],[17,18,104], [19,20,105]])
data3 = np.array([[21,22],[23,24],[25,26],[27,28], [29,30]])

y = np.array([100, 200, 300, 400, 500])
ts1 = TemporalDataStream(data1, y)
ts2 = TemporalDataStream(data2, y)
ts3 = TemporalDataStream(data3, y)

sources = (ts1, ts2, ts3)
n_streams = len(sources)
queues = [Queue()] * n_streams

condition = Condition()

producers = [Process(target=stream_producer, args=(condition, queues, sources[i], i, 2), daemon=True) for i in range(n_streams)]
for p in producers:
    p.start()

buffer_queue = Queue()        
consumer = Process(target=stream_consumer, args=(condition, queues, buffer_queue), daemon=True)
consumer.start()

experiment_queue = Queue()
experiment = Process(target=run_experiment, args=(buffer_queue, experiment_queue, n_streams), daemon=True)
experiment.start()

for p in producers:
    p.join()

for queue in queues:
    queue.put(None)
consumer.join()

buffer_queue.put(None)
while True:
    result = experiment_queue.get()
    if result is None:
        break
    else:
        print(result)
experiment_queue.put(None)
experiment.join()

print("Done")

Produced 0 = [[1 2]
 [3 4]]
Produced 2 = [[21 22]
 [23 24]]
Produced 1 = [[ 11  12 101]
 [ 13  14 102]]
Produced 2 = [[25 26]
 [27 28]]
Produced 0 = [[5 6]
 [7 8]]
Produced 1 = [[ 15  16 103]
 [ 17  18 104]]
Produced 0 = [[ 9 10]]
Produced 2 = [[29 30]]
Produced 1 = [[ 19  20 105]]


[[  1   2  11  12 101  21  22]
 [  3   4  13  14 102  23  24]]
[[  5   6  15  16 103  25  26]
 [  7   8  17  18 104  27  28]]
[  9  10  19  20 105  29  30]
Done
